In [219]:
import itertools

In [220]:
n = 2
q = int(2**n)
K = GF(q, 'a')
d = 4

In [221]:
# produces a list of q(x) with \sum n_P == 4 where Z(q(x)) = \sum n_P *P
# enumerate possibe tuples (n_P)_P
lst = []
for n_inf,n_0,n_1,n_d1,n_d2,n_d3,n_d4 in itertools.product(range(5),range(3),range(2),range(5),range(3),range(2),range(2)):
    if n_inf >= n_0 and n_0 >= n_1 and (n_inf + n_0 + n_1 + n_d1 + 2*n_d2 + 3*n_d3 + 4*n_d4 == 4):
        if n_1 == 0 and n_d1 != 0:
            continue
        # if n_d1 != 0:
        #     continue
        lst.append((n_inf,n_0,n_1,n_d1,n_d2,n_d3,n_d4))
print (lst)

[(0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 2, 0, 0), (1, 0, 0, 0, 0, 1, 0), (1, 1, 0, 0, 1, 0, 0), (1, 1, 1, 1, 0, 0, 0), (2, 0, 0, 0, 1, 0, 0), (2, 1, 1, 0, 0, 0, 0), (2, 2, 0, 0, 0, 0, 0), (3, 1, 0, 0, 0, 0, 0), (4, 0, 0, 0, 0, 0, 0)]


In [222]:
%%time

# produce a list of irreducible polynomials
R.<x> = PolynomialRing(GF(2**n, 'a'))
irred_pol = {'0':[1]}
for d in range(5):
    for pol in R.polynomials(of_degree = d):
        if pol.is_monic() is True and pol.is_irreducible() is True:
            if pol == R(x) or pol == R(x + 1):
                continue
            try:
                irred_pol[str(d)].append(pol)
            except:
                irred_pol[str(d)] = [pol]
# print(irred_pol)

CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 118 ms


In [223]:
def pol_orbits(pol,stablist):
    pol = R(pol)
    if len(stablist) == 0:
        return {pol}
    elif len(stablist) == 1 and stablist[0] == x+1:
        return {pol,pol(stablist[0])}
    elif len(stablist) == 1 and stablist[0] == 1/x:
        return {pol,pol(stablist[0])*x^7}
    elif len(stablist) == 2:
        return {pol, pol(1/x)*x^7, pol(1/(x+1))*(x+1)^7, pol(x+1), pol(1 + 1/x)*x^7, pol(x/(x+1))*(x+1)^7}

In [224]:
%%time 
# construct Q6 list from tuple (n_P)_P = (n_infty, n_0, ... , n_7)
Q6 = set()
for ele in lst:
    print(ele)
    
    # find factors of representatives
    possible_polys = {}
    for ind,count in enumerate(ele):
        if ind == 1 and count != 0:
            possible_polys['0'] = [x^count]
        elif ind == 2 and count != 0:
            possible_polys['1'] = [(x+1)^count]
        elif ind > 2 and count != 0:
            for pol_tup in itertools.product(irred_pol[str(ind-2)],repeat=int(count)):
                try:
                    possible_polys[str(ind-1)].append(prod(pol_tup))
                except:
                    possible_polys[str(ind-1)] = [prod(pol_tup)]
    polys = set([prod(i) for i in list(itertools.product(*possible_polys.values()))])


    # find stabilisers of tuple (x+1 or 1/x)
    # stab_list = []
    # if ele[1] == ele[2]:
    #     stab_list.append(x+1)
    # if ele[0] == ele[1]:
    #     stab_list.append(1/x)
    # final = polys.copy()
    # print("checking dupes", len(polys))
    
    # # check if polys are in the same equivalence class
    # for pol in polys:
    #     tmp = pol_orbits(pol,stab_list)
    #     tmp2 = [orbs for orbs in tmp if orbs in polys]
    #     if len(tmp) == 1:
    #         continue
    #     elif len(tmp) > 1 and len(tmp2) > 1:
    #         ele = tmp2[0]
    #         for pols in tmp:
    #             try:
    #                 final.remove(pols)
    #             except:
    #                 continue
    #         final.add(ele)
        
    Q6 = Q6.union(polys)
# Q6 = list(Q6)
print(len(Q6))

(0, 0, 0, 0, 0, 0, 1)
(0, 0, 0, 0, 2, 0, 0)
(1, 0, 0, 0, 0, 1, 0)
(1, 1, 0, 0, 1, 0, 0)
(1, 1, 1, 1, 0, 0, 0)
(2, 0, 0, 0, 1, 0, 0)
(2, 1, 1, 0, 0, 0, 0)
(2, 2, 0, 0, 0, 0, 0)
(3, 1, 0, 0, 0, 0, 0)
(4, 0, 0, 0, 0, 0, 0)
119
CPU times: user 6.55 ms, sys: 0 ns, total: 6.55 ms
Wall time: 6.3 ms


In [225]:
Q6 = {R(i) for i in Q6}

In [226]:
%%time
# 3) BUILD PGL₂(K) — we just take all GL₂ matrices (scalars won't change the mobius action)
GL2 = GL(2, K)
PGL2 = []
for M in GL2:
    m = M.matrix()
    if (m[0][0] == 0 and m[0][1] == 1) or m[0][0] == 1:
        PGL2.append(m)

print(len(PGL2), len(GL2))

60 180
CPU times: user 97.4 ms, sys: 0 ns, total: 97.4 ms
Wall time: 96.1 ms


In [227]:
%%time

def mobius(pol,g):
    # level = d
    # g = g.matrix()
    # print(((g[1][0]*x + g[1][1])^(level)*pol((g[0][0]*x + g[0][1])/(g[1][0]*x + g[1][1]))))
    return ((g[1][0]*x + g[1][1])^(d)*pol((g[0][0]*x + g[0][1])/(g[1][0]*x + g[1][1]))).numerator().monic()

# 5) Approximate 2-pass collapse to make faster.
for cutoff in [10, 50]:
    reps = Q6.copy()
    for i, M in enumerate(PGL2):
        if i%20 == 0:
            print(i, len(reps))
        l = len(reps)
        temp = reps.copy()
        for j, q in enumerate(reps):
            if j == min(cutoff, int(l*0.2)) and l - len(temp) == 0:
                break
            if q in temp:
                r = mobius(q, M)
                # if r == 1:
                    # print(q, r, M, r in temp, r != q)
                if r in temp and r != q:
                    temp.remove(r)
        reps = temp
    Q6 = reps
    print(len(Q6))
    # print(Q6)

0 119
20 28
40 18
17
0 17
20 17
40 17
17
CPU times: user 702 ms, sys: 8 ms, total: 710 ms
Wall time: 707 ms


In [228]:
# 5) Final collapse, precise computation.
reps = []
seen = set()
for i, q in enumerate(Q6):
    print(i)
    if q in seen:
        continue
    if q == 1:
        seen.add(q)
        reps.append(q)
        # print("1 appended")
        continue
    # compute entire orbit under PGL₂
    orb = {mobius(q, M) for M in PGL2}
    # choose the smallest in lex order on coefficient‐vectors
    rep = min(orb, key=lambda p: tuple(p.coefficients(sparse=False)))
    reps.append(rep)
    seen.update(orb)
    

print(f"After modding out by PGL₂, you get {len(reps)} representatives Q := {{q₁,…,qₙ}}.")
# `reps` is now exactly the list of q(x) up to isomorphism.
print(reps)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
After modding out by PGL₂, you get 15 representatives Q := {q₁,…,qₙ}.
[1, x^4 + x^3 + 1, x^4 + (a + 1)*x^3 + x^2 + 1, x^4 + a*x^3 + x^2 + 1, x^4 + x, x^3, x^4 + a*x, x^4 + x^3 + x, x^4 + a*x^3 + x^2 + x, x^4 + a*x^2 + 1, x^4 + a*x^3 + x^2, x^4 + (a + 1)*x^3 + x^2 + x, x^3 + x^2, x^4 + (a + 1)*x^3 + x^2 + a*x, x^2]


In [229]:
# print(Q6)
Q6 = reps

In [230]:
from collections import defaultdict
F = GF(2**n, 'a')

Q3 = [R(i) for i in Q6]

F.<a> = GF(2**n)
R.<x> = PolynomialRing(F)
S.<x> = R.quotient(x^9)
G = GL(2, F)
G_lst = list(G.list())
# G_lst = PGL2
def g_action(level, g, P):
    [[a,b], [c,d]] = g.list()
    A, B = a*x + b, c*x + d
    coeffs = P.list() + [0]*(level+1 - len(P.list()))
    result = sum(p_i * A**i * B**(level - i)
            for i, p_i in enumerate(coeffs) if p_i)
    return result

basis = [a^i * S(x^j) for i in range(n) for j in range(9)]
V = VectorSpace(GF(2), 9*n)

def basechange(f):
    if f == 1:
        return [1] + [0] * (9*n - 1)
    coeffs = f.list()
    coeffs += [0] * (9 - len(coeffs))
    v = [0]*(9*n)
    for exp, c in enumerate(coeffs):
        rep = c.polynomial().list()
        rep += [0] * (n - len(rep))
        for i, bit in enumerate(rep):
            if bit:
                v[i*9 + exp] = 1
    return v


Vq = defaultdict(list)
print(len(Q3))
stabilizers = []
for q in Q3:
    print(q)
    basis_PhiV = [a^j * x^i * q + a^(2*j) * x^(2*i) for j in range(n) for i in range(5)]
    new_basis_PhiV = [basechange(f) for f in basis_PhiV]
    PhiV = V.subspace([V(base) for base in new_basis_PhiV])
    Quotient = V.quotient(PhiV)
    candidates = set()
    for coset in Quotient:
        vec = tuple(Quotient.lift(coset))
        p = sum(bit * b for bit, b in zip(vec, basis))
        candidates.add(p)
    Rq = R(q)
    stab_q = [A for A in GL2 if g_action(4, A, q) == q]
    stabilizers.append((q, stab_q))
    print(len(candidates), len(stab_q))
    while candidates:
        p = candidates.pop()
        if p == 0:
            p_deg = -1
        else:
            p_deg = max(i for i, c in enumerate(p.list()) if c!=0)
        m = max(2*q.degree(), p_deg)

        if not m in [7,8]:
            continue

        orbit = set()
        for A in stab_q:
            fS = S(g_action(8, A, p))
            vec_f = basechange(fS)
            cos_f = Quotient(V(vec_f))
            lift_f = tuple(Quotient.lift(cos_f))
            v = sum(bit * b for bit, b in zip(lift_f, basis))
            orbit.add(v)
                
        candidates -= orbit
        Vq.setdefault(Rq, []).append(p)

hyperelliptics = {}
def smooth(p,q):
    if not isinstance(p, int):
        p_lift = S.lift(p)
        p_deriv = p_lift.derivative()
    else:
        p_lift = p
        p_deriv = 0
    return gcd(q, p_deriv^2 + (q.derivative()^2)*p_lift) == 1 and (q.degree() == 4 or p_lift.monomial_coefficient(S.lift(x^7))^2 != p_lift.monomial_coefficient(S.lift(x^8))*q.monomial_coefficient(S.lift(x^3))^2)
for q in Vq:
    p_list = []
    for p in Vq[q]:
        if smooth(p,q):
            p_list.append(p)
    hyperelliptics[q] = p_list

final_hyp = hyperelliptics
print(sum([len(l) for l in final_hyp.values()]))

15
1
512 12
x^4 + x^3 + 1
512 4
x^4 + (a + 1)*x^3 + x^2 + 1
512 2
x^4 + a*x^3 + x^2 + 1
512 2
x^4 + x
512 12
x^3
512 3
x^4 + a*x
512 3
x^4 + x^3 + x
512 1
x^4 + a*x^3 + x^2 + x
512 2
x^4 + a*x^2 + 1
512 10
x^4 + a*x^3 + x^2
512 2
x^4 + (a + 1)*x^3 + x^2 + x
512 2
x^3 + x^2
512 2
x^4 + (a + 1)*x^3 + x^2 + a*x
512 3
x^2
512 6
2162


In [231]:
hyperelliptics = {}
def smooth(p,q):
#   p = R(p.list())
#   q = R(q.list())
    if not isinstance(p, int):
       # p_lift = S.lift(p)
        p = R(p.list())
        q = R(q.list())
        p_deriv = p.derivative()
    else:
        p = R(p)
        q = R(q)
        p_deriv = 0
    x = R.gen()
    return (gcd(q, p.derivative()^2 + (q.derivative()^2)*p) == 1 
                and (q.degree() == 4 
                or p.monomial_coefficient(R(x^7))^2 != p.monomial_coefficient(R(x^8))*q.monomial_coefficient(R(x^3))^2))
    

for q in Vq:
    p_list = []
    for p in Vq[q]:
        if smooth(p,q):
            p_list.append(p)
    hyperelliptics[q] = p_list

final_hyp = hyperelliptics
print(sum([len(l) for l in final_hyp.values()]))

2162


In [232]:
import os
print(sum(len(x) for x in final_hyp.values()))
num_blocks = 64
files = []
for i in range(num_blocks):
    files.append(open(os.path.join("data"+str(2**n), "block_"+str(i)), 'a+'))
i = 0
for q in final_hyp:
    print(q)
    for p in final_hyp[q]:
        files[i%num_blocks].write(str(q.list()) + ';' + str(p.list()) + '\n')
        i += 1

for file in files:
    file.close()

2162


FileNotFoundError: [Errno 2] No such file or directory: 'data4/block_0'

In [ ]:
all_hyp = []
for q in final_hyp:
    for p in final_hyp[q]:
        all_hyp.append((q, p))
# all_hyp = [(q, p) for p in final_hyp[q] for q in final_hyp]
print(len(all_hyp))
out = defaultdict(int)
Ring.<x> = PolynomialRing(GF(2**n, 'a'))

for i, (q, p) in enumerate(all_hyp):
    # print(q, p, type(p), type(q))
    if i % 200 == 0:
        print(i)
    C = HyperellipticCurve(Ring(p.list()), Ring(q.list()))
    frob = C.frobenius_polynomial()
    out[frob] += 1

print(len(out))
# print(out)

66514
0
200


KeyboardInterrupt: 

In [ ]:
sum(out.values())

4178

In [ ]:
# Q6 = [(z4^3 + z4^2 + z4 + 1)*x^8 + (z4^3 + z4^2 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, 1, (z4^2 + z4)*x^8 + z4^2*x^7 + x^5 + x^4, (z4^3 + 1)*x^8 + z4^2*x^7 + x^6 + x^4, (z4^3 + z4^2 + z4)*x^8 + (z4^3 + z4^2 + z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2)*x^8 + (z4 + 1)*x^7 + (z4^3 + z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + x^7 + z4^3*x^6 + x^5 + x^4, (z4^3 + z4 + 1)*x^4 + x^3 + x^2, x^8 + (z4^2 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, z4^2*x^8 + (z4^3 + z4^2 + z4 + 1)*x^7 + x^6 + x^4, z4^3*x^8 + x^6 + x^4, (z4^3 + z4)*x^8 + (z4^3 + z4^2 + z4 + 1)*x^7 + z4*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^6 + x^5 + x^3, z4^3*x^8 + (z4^2 + z4)*x^7 + x^5 + x^4, z4*x^8 + (z4^3 + z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^2 + 1)*x^8 + (z4^3 + z4 + 1)*x^7 + x^5 + x^4, (z4^3 + z4^2 + z4 + 1)*x^8 + (z4^3 + z4^2 + 1)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, x^8 + (z4^2 + 1)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4)*x^8 + (z4^3 + z4^2 + 1)*x^7 + z4^3*x^6 + x^5 + x^4, x, x^2, (z4^2 + z4)*x^8 + (z4 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4 + 1)*x^8 + (z4^2 + z4 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^2 + z4 + 1)*x^8 + (z4^2 + z4)*x^7 + z4^2*x^6 + x^5 + x^4, z4^3*x^8 + (z4^2 + z4)*x^7 + z4^2*x^6 + x^5 + x^4, (z4 + 1)*x^8 + (z4 + 1)*x^7 + z4^2*x^6 + x^5 + x^4, (z4^2 + 1)*x^8 + (z4^3 + z4 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4 + 1)*x^8 + (z4^3 + z4)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4 + 1)*x^8 + x^7 + z4^3*x^6 + x^5 + x^4, z4*x^8 + (z4^2 + z4)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^2 + 1)*x^8 + x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4 + 1)*x^6 + (z4^2 + 1)*x^5 + x^3, (z4^3 + z4^2 + z4 + 1)*x^8 + x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, z4^2*x^8 + (z4^2 + z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4 + 1)*x^6 + z4*x^5 + x^3, z4^3*x^8 + (z4^3 + z4 + 1)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2)*x^8 + (z4^2 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^3 + z4^2)*x^8 + (z4^2 + z4)*x^7 + z4*x^6 + x^5 + x^4, (z4^3 + z4^2)*x^6 + (z4 + 1)*x^5 + x^3, (z4^3 + z4^2 + z4 + 1)*x^8 + (z4^3 + z4^2 + 1)*x^7 + z4^3*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + (z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4 + 1)*x^8 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4 + 1)*x^8 + (z4^3 + z4)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, z4*x^8 + (z4^2 + z4 + 1)*x^7 + x^6 + x^4, (z4^2 + z4)*x^6 + (z4 + 1)*x^5 + x^3, (z4^3 + 1)*x^8 + (z4^3 + z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, z4^3*x^8 + (z4^3 + z4^2 + z4)*x^7 + x^5 + x^4, z4*x^8 + z4^3*x^7 + x^6 + x^4, (z4^3 + z4^2 + z4)*x^6 + z4^2*x^5 + x^3, x^8 + (z4^3 + z4^2)*x^7 + z4^2*x^6 + x^5 + x^4, (z4^3 + z4 + 1)*x^8 + (z4^3 + z4^2 + z4)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^3 + z4)*x^8 + (z4^3 + z4 + 1)*x^7 + z4^3*x^6 + x^5 + x^4, (z4^2 + z4 + 1)*x^8 + (z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4 + 1)*x^8 + (z4^2 + 1)*x^7 + x^6 + x^4, (z4^2 + z4)*x^8 + (z4 + 1)*x^7 + z4^2*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4 + 1)*x^8 + (z4^3 + z4^2)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4 + 1)*x^8 + z4^3*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^2 + z4)*x^8 + x^7 + x^5 + x^4, (z4^3 + 1)*x^8 + (z4^3 + 1)*x^7 + (z4^3 + z4^2 + z4)*x^6 + x^5 + x^4, z4^3*x^8 + (z4^3 + z4^2 + 1)*x^7 + (z4^3 + z4^2 + z4)*x^6 + x^5 + x^4, z4*x^8 + (z4^3 + 1)*x^7 + x^5 + x^4, z4*x^8 + (z4^3 + z4 + 1)*x^7 + z4^3*x^6 + x^5 + x^4, z4^3*x^8 + (z4^3 + 1)*x^7 + x^5 + x^4, (z4^2 + 1)*x^8 + x^7 + x^4, (z4^3 + z4^2 + z4)*x^8 + (z4^2 + z4)*x^7 + (z4^3 + z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + (z4^3 + z4^2 + 1)*x^7 + z4*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4)*x^8 + (z4^2 + 1)*x^7 + z4*x^6 + x^5 + x^4, (z4^2 + 1)*x^8 + (z4^3 + z4^2 + z4 + 1)*x^7 + x^5 + x^4, (z4^2 + z4)*x^8 + (z4^3 + 1)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + z4*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, z4*x^8 + (z4^3 + z4 + 1)*x^7 + z4^2*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + z4^3*x^7 + z4^3*x^6 + x^5 + x^4, (z4^3 + 1)*x^6 + (z4^2 + 1)*x^5 + x^3, (z4^3 + z4^2 + 1)*x^8 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^6 + z4*x^5 + x^3, (z4^3 + z4^2 + 1)*x^8 + (z4^2 + 1)*x^7 + z4*x^6 + x^5 + x^4, (z4 + 1)*x^8 + (z4^3 + z4)*x^7 + x^6 + x^4, z4^2*x^6 + x^3, z4*x^8 + (z4 + 1)*x^7 + x^5 + x^4, (z4 + 1)*x^6 + x^3, (z4^3 + z4 + 1)*x^8 + z4^3*x^7 + (z4^3 + z4^2 + z4)*x^6 + x^5 + x^4, (z4^2 + z4)*x^8 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^2 + z4)*x^8 + (z4^3 + z4^2)*x^6 + x^5 + x^4, z4*x^8 + (z4^3 + z4^2)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4)*x^8 + (z4^2 + z4)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4)*x^8 + z4^2*x^7 + z4^2*x^6 + x^5 + x^4, z4^2*x^6 + (z4 + 1)*x^5 + x^3, (z4^3 + z4)*x^6 + z4^2*x^5 + x^3, (z4^3 + z4^2)*x^6 + (z4^2 + 1)*x^5 + x^3, (z4^3 + z4^2 + 1)*x^8 + x^7 + z4^2*x^6 + x^5 + x^4, (z4^3 + z4)*x^8 + (z4^2 + z4)*x^7 + x^6 + x^4, (z4^2 + 1)*x^8 + z4*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, x^8 + (z4^3 + z4)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + (z4^3 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^3 + z4)*x^8 + z4^2*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + z4*x^7 + x^5 + x^4, (z4^3 + 1)*x^8 + (z4^3 + z4^2 + z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, z4*x^8 + (z4^3 + z4^2)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4 + 1)*x^6 + (z4 + 1)*x^5 + x^3, z4*x^8 + (z4^3 + z4^2)*x^7 + x^6 + x^4, (z4^3 + 1)*x^8 + (z4^3 + z4^2)*x^7 + z4^3*x^6 + x^5 + x^4, (z4^2 + 1)*x^6 + z4^2*x^5 + x^3, (z4^3 + z4^2)*x^8 + (z4^3 + z4^2 + z4 + 1)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, z4^3*x^8 + (z4^2 + z4 + 1)*x^7 + x^6 + x^4, z4^3*x^8 + (z4^3 + z4^2)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, (z4^3 + z4 + 1)*x^8 + (z4 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4 + 1)*x^8 + (z4^2 + z4 + 1)*x^7 + x^6 + x^4, (z4^3 + 1)*x^8 + (z4^3 + z4 + 1)*x^7 + z4^3*x^6 + x^5 + x^4, z4^2*x^8 + x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4 + 1)*x^8 + (z4^3 + 1)*x^7 + x^5 + x^4, (z4 + 1)*x^8 + z4^3*x^7 + z4*x^6 + x^5 + x^4, z4^2*x^8 + (z4^2 + z4)*x^7 + x^5 + x^4, (z4^3 + z4^2 + 1)*x^8 + (z4^2 + z4)*x^7 + (z4^3 + z4^2 + z4)*x^6 + x^5 + x^4, z4^3*x^8 + (z4^3 + z4)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, (z4^2 + z4 + 1)*x^8 + (z4^3 + z4 + 1)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4 + 1)*x^8 + (z4^2 + 1)*x^7 + z4^2*x^6 + x^5 + x^4, x^6 + x^3, (z4^3 + 1)*x^8 + z4^2*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + 1)*x^8 + (z4^2 + z4)*x^7 + x^6 + x^4, z4^3*x^8 + (z4^2 + z4 + 1)*x^7 + (z4^3 + z4)*x^6 + x^5 + x^4, z4^3*x^6 + (z4 + 1)*x^5 + x^3, z4^2*x^8 + (z4 + 1)*x^7 + x^6 + x^4, x^8 + (z4 + 1)*x^7 + z4^2*x^6 + x^5 + x^4, z4^2*x^8 + (z4 + 1)*x^7 + (z4^2 + z4)*x^6 + x^5 + x^4, (z4^3 + z4)*x^8 + (z4^2 + 1)*x^7 + (z4^3 + z4^2)*x^6 + x^5 + x^4, z4*x^6 + (z4 + 1)*x^5 + x^3, (z4^3 + 1)*x^8 + z4*x^7 + x^6 + x^4, (z4^3 + z4)*x^8 + (z4^3 + 1)*x^7 + z4^2*x^6 + x^5 + x^4, (z4^3 + z4^2 + z4)*x^8 + (z4^2 + 1)*x^7 + z4^3*x^6 + x^5 + x^4, z4^3*x^8 + (z4^2 + z4)*x^7 + x^6 + x^4]

In [1]:
%%cython
from collections import defaultdict
from sage.all import GF, PolynomialRing, GL, VectorSpace, gcd, prod
import itertools

def gen_qs(n):
    K = GF(2**n, 'a')
    a = K.gen()
    lst = []
    for n_inf,n_0,n_1,n_d1,n_d2,n_d3,n_d4 in itertools.product(range(5),range(3),range(2),range(5),range(3),range(2),range(2)):
        if n_inf >= n_0 and n_0 >= n_1 and (n_inf + n_0 + n_1 + n_d1 + 2*n_d2 + 3*n_d3 + 4*n_d4 == 4):
            if n_1 == 0 and n_d1 != 0:
                continue
            lst.append((n_inf,n_0,n_1,n_d1,n_d2,n_d3,n_d4))
    R = PolynomialRing(K, 'x')
    x = R.gen()
    irred_pol = {'0':[1]}
    for d in range(5):
        for pol in R.polynomials(of_degree = d, is_monic=True):
            if pol.is_monic() is True and pol.is_irreducible() is True:
                if pol == R(x) or pol == R(x + 1):
                    continue
                try:
                    irred_pol[str(d)].append(pol)
                except:
                    irred_pol[str(d)] = [pol]

    Q6 = set()
    for ele in lst:
        print(ele)
        possible_polys = {}
        for ind,count in enumerate(ele):
            if ind == 1 and count != 0:
                possible_polys['0'] = [x**count]
            elif ind == 2 and count != 0:
                possible_polys['1'] = [(x+1)**count]
            elif ind > 2 and count != 0:
                for pol_tup in itertools.product(irred_pol[str(ind-2)],repeat=int(count)):
                    try:
                        possible_polys[str(ind-1)].append(prod(pol_tup))
                    except:
                        possible_polys[str(ind-1)] = [prod(pol_tup)]
        polys = set([prod(i) for i in list(itertools.product(*possible_polys.values()))])
        Q6 = Q6.union(polys)

    Q6 = {R(i) for i in Q6}
    GL2, PGL2 = GL(2, K), []
    for M in GL2:
        m = M.matrix()
        if m[0][0] == 1 or (m[0][0] == 0 and m[1][0] == 1):
            PGL2.append(m)

    def mobius(pol,g):
        return ((g[1][0]*x + g[1][1])**(d)*pol((g[0][0]*x + g[0][1])/(g[1][0]*x + g[1][1]))).numerator().monic()

    for cutoff in [10, 50]:
        reps = Q6.copy()
        for i, M in enumerate(PGL2):
            if i%20 == 0:
                print(i, len(reps))
            l = len(reps)
            temp = reps.copy()
            for j, q in enumerate(reps):
                if j == min(cutoff, int(l*0.2)) and l - len(temp) == 0:
                    break
                if q in temp:
                    r = mobius(q, M)
                    if r == 1:
                        print(q, r, M, r in temp, r != q)
                    if r in temp and r != q:
                        temp.remove(r)
            reps = temp
        Q6 = reps
    reps = []
    seen = set()
    for i, q in enumerate(Q6):
        print(i)
        if q in seen:
            continue
        if q == 1:
            seen.add(q)
            reps.append(q)
            print("1 appended")
            continue
        orb = {mobius(q, M) for M in PGL2}
        rep = min(orb, key=lambda p: tuple(p.coefficients(sparse=False)))
        reps.append(rep)
        seen.update(orb)

    Q6 = reps
    return Q6


def g3_hyp_even_5(n):
    Q3 = gen_qs(n)
    F = GF(2**n, 'a')
    a = F.gen()
    R = PolynomialRing(F, 'x')
    x = R.gen()
    S = R.quotient(x**9, names='x')
    x = S.gen()
    G = GL(2, F)
    PGL = [A for A in G if (A.list()[0][0]==1 or (A.list()[0][0] == 0 and A.list()[1][0] == 1))]
    # G_lst = list(G.list())
    def g_action(level, g, P):
        [[a,b], [c,d]] = g.list()
        A, B = a*x + b, c*x + d
        coeffs = P.list() + [0]*(level+1 - len(P.list()))
        result = sum(p_i * A**i * B**(level - i)
                for i, p_i in enumerate(coeffs) if p_i)
        return result

    basis = [a**i * S(x**j) for i in range(n) for j in range(9)]
    V = VectorSpace(GF(2), 9*n)
    print('step 4')

    def basechange(f):
        if f == 1:
            return [1] + [0] * (9*n - 1)
        coeffs = f.list()
        coeffs += [0] * (9 - len(coeffs))
        v = [0]*(9*n)
        for exp, c in enumerate(coeffs):
            rep = c.polynomial().list()
            rep += [0] * (n - len(rep))
            for i, bit in enumerate(rep):
                if bit:
                    v[i*9 + exp] = 1
        return v

    Vq = defaultdict(list)
    print(len(Q3))
    for q in Q3:
        print(q)
        basis_PhiV = [a**j * x**i * q + a**(2*j) * x**(2*i) for j in range(n) for i in range(5)]
        new_basis_PhiV = [basechange(f) for f in basis_PhiV]
        PhiV = V.subspace([V(base) for base in new_basis_PhiV])
        Quotient = V.quotient(PhiV)
        candidates = set()
        for coset in Quotient:
            vec = tuple(Quotient.lift(coset))
            p = sum(bit * b for bit, b in zip(vec, basis))
            candidates.add(p)
        Rq = R(q)
        stab_q = [A for A in G if g_action(4, A, q) == q]
        while candidates:
            p = candidates.pop()
            if p == 0:
                p_deg = -1
            else:
                p_deg = max(i for i, c in enumerate(p.list()) if c!=0)
            m = max(2*q.degree(), p_deg)

            if not m in [7,8]:
                continue

            orbit = set()
            for A in stab_q:
                fS = S(g_action(8, A, p))
                vec_f = basechange(fS)
                cos_f = Quotient(V(vec_f))
                lift_f = tuple(Quotient.lift(cos_f))
                v = sum(bit * b for bit, b in zip(lift_f, basis))
                orbit.add(v)
            candidates -= orbit
            Vq.setdefault(Rq, []).append(p)

    hyperelliptics = {}
    def smooth(p,q):
        if not isinstance(p, int):
            p_lift = S.lift(p)
            p_deriv = p_lift.derivative()
        else:
            p_lift = p
            p_deriv = 0
        return q.gcd(p_deriv**2 + (q.derivative()**2)*p_lift) == 1 and (q.degree() == 4 or p_lift.monomial_coefficient(S.lift(x**7))**2 != p_lift.monomial_coefficient(S.lift(x**8))*q.monomial_coefficient(S.lift(x**3))**2)
    print('step 5')
    for q in Vq:
        p_list = []
        for p in Vq[q]:
            if smooth(p,q):
                p_list.append(p)
        hyperelliptics[q] = p_list

    return hyperelliptics


In [2]:
%%time 
result = g3_hyp_even_5(2)
print(sum(len(r) for r in result.values()))

TypeError: PolynomialRing_general.polynomials() got an unexpected keyword argument 'is_monic'

In [235]:
import os
final_hyp = result
with open("16", 'w') as file:
    for q in final_hyp:
        print(q)
        for p in final_hyp[q]:
            file.write(str(q.list()) + ';' + str(p.list()) + '\n')
            i += 1


1
x^4 + a*x^3 + x^2
x^4 + (a^3 + a)*x^3 + x^2 + x
x^3 + x^2
x^4 + (a^3 + a^2 + a + 1)*x^3 + x^2 + x
x^4 + (a^3 + a^2 + 1)*x^3 + a*x^2 + 1
x^4 + a*x^3 + x
x^4 + (a^2 + 1)*x^3 + x^2 + x
x^4 + (a^2 + a + 1)*x^3 + x^2 + 1
x^4 + (a^2 + 1)*x^3 + a^2*x^2 + 1
x^4 + a^2*x^3 + (a^2 + 1)*x^2 + 1
x^4 + a^3*x^3 + x^2 + 1
x^4 + a^3*x^3 + x^2 + x
x^4 + (a^3 + a^2 + a)*x^3 + a*x^2 + 1
x^4 + (a^3 + a^2)*x^3 + x^2 + 1
x^4 + x^3 + x
x^4 + a^2*x^3 + x^2 + x
x^4 + (a^2 + a)*x^3 + x^2 + x
x^4 + (a^3 + a + 1)*x^3 + (a + 1)*x^2 + 1
x^4 + (a^3 + a^2)*x^3 + x^2 + x
x^4 + a^2*x^3 + x
x^4 + (a + 1)*x^3 + a^2*x^2 + x
x^4 + a*x^3 + x^2 + x
x^4 + a*x^2 + 1
x^4 + a^3*x^3 + 1
x^4 + x
x^4 + (a^3 + a)*x^3 + x^2 + 1
x^4 + (a + 1)*x^3 + x^2 + x
x^4 + (a + 1)*x^3 + x
x^4 + a*x^3 + (a + 1)*x^2 + 1
x^4 + (a^3 + 1)*x^3 + (a + 1)*x^2 + 1
x^4 + (a^2 + 1)*x^3 + x
x^4 + (a^3 + a^2 + a + 1)*x^3 + x^2 + 1
x^4 + (a^2 + a)*x^3 + x^2 + 1
x^4 + (a^2 + a + 1)*x^3 + x^2 + x
x^4 + a^2*x^3 + a*x^2 + x
x^4 + (a + 1)*x^3 + a*x^2 + 1
x^3
x^2
